In [1]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [49]:
df = pd.read_csv('product.csv')

In [50]:
df.head()

,UTM_Source,UTM_Medium,Trans_ID,Loan_id,uuid,Opened_date,Closed_date,Product,"$43,484.00","$43,515.00",...,"$43,604.00","$43,635.00","$43,665.00","$43,696.00","$43,727.00","$43,757.00","$43,788.00","$43,818.00","$43,485.00",TotalRevenueTodate
0,NaN,NaN,NaN,26,704ddfea-ba01-4560-8aef-1550932d04d7,1/3/19,NaN,Prime12,$8.95,$10.46,...,$7.73,$7.14,$2.87,$5.42,$4.54,$4.01,$0,$9.16,NaN,$72.11
1,NaN,NaN,NaN,27,df007477-d209-445a-80e3-649cd1e301d0,1/8/19,NaN,Basic12,$8.95,$5.25,...,$3.76,$7.46,$6.96,$3.07,$2.10,$0,$4.91,$2.89,NaN,$39.89
2,NaN,NaN,NaN,28,3066d9e8-9575-4fac-88b9-aa0ce2cafc3e,1/10/19,3/26/19,Prime12,$8.95,$15.19,...,$0,$0,$8.95,NaN,NaN,NaN,NaN,NaN,NaN,$27.84
3,NaN,NaN,NaN,29,9ef9086b-d671-48f6-a490-9f98152c4ea5,1/11/19,NaN,Prime24,$8.95,$11.86,...,$10.22,$10.12,$0.41,$9.21,$8.75,$9.62,$7.83,$7.11,NaN,$105.43
4,NaN,NaN,NaN,30,551f0be6-6264-4636-a864-2da25babe77d,1/14/19,2/12/19,Basic24,$0,$0,...,$0,$0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$0


In [54]:
#reduce columns
df = df[['uuid','Opened_date','Product','TotalRevenueTodate']]

In [55]:
#rename columns
df.columns = ['uuid','Opened_date','Product','Revenue']

In [56]:
#change opened date
df['Opened_date'] = pd.to_datetime(df['Opened_date'])

In [57]:
#check for missing data
df.isnull().sum()

uuid           0
Opened_date    0
Product        0
Revenue        0
dtype: int64

In [46]:
df['Revenue'] = df['Revenue'].replace({'\$': '', ',': ''}, regex=True).astype(float)

In [59]:
df.head()

,uuid,Opened_date,Product,Revenue
0,704ddfea-ba01-4560-8aef-1550932d04d7,2019-01-03,Prime12,72.11
1,df007477-d209-445a-80e3-649cd1e301d0,2019-01-08,Basic12,39.89
2,3066d9e8-9575-4fac-88b9-aa0ce2cafc3e,2019-01-10,Prime12,27.84
3,9ef9086b-d671-48f6-a490-9f98152c4ea5,2019-01-11,Prime24,105.43
4,551f0be6-6264-4636-a864-2da25babe77d,2019-01-14,Basic24,0.00


# RFM Model

In [60]:
import datetime as dt
NOW = dt.datetime(2019,12,20)

In [63]:
#rfm model
rfmTable = df.groupby('uuid').agg({'Opened_date': lambda x: (NOW - x.max()).days,
                                        'uuid': lambda x: len(x),  
                                        'Revenue': lambda x: x.sum()})

rfmTable['Opened_date'] = rfmTable['Opened_date'].astype(int)
rfmTable.rename(columns={'Opened_date': 'recency', 
                         'uuid': 'frequency', 
                         'Revenue': 'monetary_value'}, inplace=True)
rfmTable.head()

,recency,frequency,monetary_value
uuid,,,
0000e106-72fc-4b9d-b0f7-2c54a67982c3,18,1,0.00
000acd0f-e9cf-4a71-8069-28cc381c4b2b,79,1,0.00
000d12ab-88aa-437c-bb75-3d65ab8767a6,109,1,76.12
0018f6d4-ed83-4ac3-a5d9-97e82e15bebd,89,1,37.41
001a2dd4-7ab7-4953-b141-11e3ac706611,49,1,45.72


In [64]:
#split quantiles
quantiles = rfmTable.quantile(q=[0.2,0.4,0.6,0.8])
quantiles = quantiles.to_dict()

In [65]:
def RScore(x,p,d):
    if x <= d[p][0.20]:
        return 1
    elif x <= d[p][0.40]:
        return 2
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 4
    else:
        return 5
    
def FMScore(x,p,d):
    if x <= d[p][0.20]:
        return 5
    elif x <= d[p][0.40]:
        return 4
    elif x <= d[p][0.60]: 
        return 3
    elif x <= d[p][0.80]: 
        return 2
    else:
        return 1

In [67]:
segmented_rfm = rfmTable

In [68]:
#segment the data
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(FMScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(FMScore, args=('monetary_value',quantiles,))
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile
uuid,,,,,,
0000e106-72fc-4b9d-b0f7-2c54a67982c3,18,1,0.00,1,5,5
000acd0f-e9cf-4a71-8069-28cc381c4b2b,79,1,0.00,3,5,5
000d12ab-88aa-437c-bb75-3d65ab8767a6,109,1,76.12,3,5,1
0018f6d4-ed83-4ac3-a5d9-97e82e15bebd,89,1,37.41,3,5,2
001a2dd4-7ab7-4953-b141-11e3ac706611,49,1,45.72,2,5,2


In [70]:
#add rfm
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)
segmented_rfm.head()

,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
uuid,,,,,,,
0000e106-72fc-4b9d-b0f7-2c54a67982c3,18,1,0.00,1,5,5,155
000acd0f-e9cf-4a71-8069-28cc381c4b2b,79,1,0.00,3,5,5,355
000d12ab-88aa-437c-bb75-3d65ab8767a6,109,1,76.12,3,5,1,351
0018f6d4-ed83-4ac3-a5d9-97e82e15bebd,89,1,37.41,3,5,2,352
001a2dd4-7ab7-4953-b141-11e3ac706611,49,1,45.72,2,5,2,252


In [71]:
#rfm output
from datetime import date
from datetime import datetime
from datetime import timedelta 

current_time = datetime.now()
info = current_time.strftime('%m%d%Y')
output_filename = ("rfm_" + str(info[:4]) + str(info[6:]) + ".csv")
segmented_rfm.to_csv(output_filename, sep=',', header=True)

# LTV Model

In [72]:
segmented_rfm2 = segmented_rfm[['recency','frequency','monetary_value']]

In [73]:
#rename columns
segmented_rfm2.columns=['num_days','num_transactions','spent_money']
segmented_rfm2.head()

,num_days,num_transactions,spent_money
uuid,,,
0000e106-72fc-4b9d-b0f7-2c54a67982c3,18,1,0.00
000acd0f-e9cf-4a71-8069-28cc381c4b2b,79,1,0.00
000d12ab-88aa-437c-bb75-3d65ab8767a6,109,1,76.12
0018f6d4-ed83-4ac3-a5d9-97e82e15bebd,89,1,37.41
001a2dd4-7ab7-4953-b141-11e3ac706611,49,1,45.72


In [74]:
#CLTV formula
#CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.
#Customer Value = Average Order Value * Purchase Frequency

In [75]:
# Average Order Value
segmented_rfm2['avg_order_value']=segmented_rfm2['spent_money']/segmented_rfm2['num_transactions']

In [76]:
purchase_frequency=sum(segmented_rfm2['num_transactions'])/segmented_rfm2.shape[0]

In [77]:
repeat_rate=segmented_rfm2[segmented_rfm2.num_transactions > 1].shape[0]/segmented_rfm2.shape[0]

In [78]:
churn_rate=1-repeat_rate

In [79]:
#assume profit margin is 5%
segmented_rfm2['profit_margin']=1

In [80]:
# Customer Value
segmented_rfm2['CLV']=(segmented_rfm2['avg_order_value']*purchase_frequency)/churn_rate

In [81]:
#Customer Lifetime Value
segmented_rfm2['cust_lifetime_value']=segmented_rfm2['CLV']*segmented_rfm2['profit_margin']

In [82]:
#output ltv model
current_time = datetime.now()
info = current_time.strftime('%m%d%Y')
output_filename = ("ltv_" + str(info[:4]) + str(info[6:]) + ".csv")
segmented_rfm2.to_csv(output_filename, sep=',', header=True)

# Cohort Analysis

In [83]:
#cohort is a group of users sharing a particular characteristic. 
#Strictly speaking it can be any characteristic, but typically the term cohort refers to a time-dependent grouping. 
#For example, a typical cohort groups users by the week or month when they were first acquired. 
#When speaking of groupings that are not time-dependent, the term segment is typically used instead of cohort.
#A cohort analysis refers to tracking and investigating the performance of cohorts over time.

In [84]:
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [92]:
#Create a period column based on the giftDate
df['Period'] = df.Opened_date.apply(lambda x: x.strftime('%M'))
df.head()

,uuid,Opened_date,Product,Revenue,Period,CohortGroup
0,704ddfea-ba01-4560-8aef-1550932d04d7,2019-01-03,Prime12,72.11,00,00
1,df007477-d209-445a-80e3-649cd1e301d0,2019-01-08,Basic12,39.89,00,00
2,3066d9e8-9575-4fac-88b9-aa0ce2cafc3e,2019-01-10,Prime12,27.84,00,00
3,9ef9086b-d671-48f6-a490-9f98152c4ea5,2019-01-11,Prime24,105.43,00,00
4,551f0be6-6264-4636-a864-2da25babe77d,2019-01-14,Basic24,0.00,00,00


In [93]:
#Determine the user's cohort group (based on their first order)
df.set_index('uuid', inplace=True)

df['CohortGroup'] = df.groupby(level=0)['Opened_date'].min().apply(lambda x: x.strftime('%D'))
df.reset_index(inplace=True)
df.head()

,uuid,Opened_date,Product,Revenue,Period,CohortGroup
0,704ddfea-ba01-4560-8aef-1550932d04d7,2019-01-03,Prime12,72.11,00,01/03/19
1,df007477-d209-445a-80e3-649cd1e301d0,2019-01-08,Basic12,39.89,00,01/08/19
2,3066d9e8-9575-4fac-88b9-aa0ce2cafc3e,2019-01-10,Prime12,27.84,00,01/10/19
3,9ef9086b-d671-48f6-a490-9f98152c4ea5,2019-01-11,Prime24,105.43,00,01/11/19
4,551f0be6-6264-4636-a864-2da25babe77d,2019-01-14,Basic24,0.00,00,01/14/19


In [90]:
#Rollup data by CohortGroup & OrderPeriod
grouped = df.groupby(['CohortGroup', 'Period'])

# count the unique users, gifttype*, and gift amount + Period
cohorts = grouped.agg({'uuid': pd.Series.nunique,
                       'Revenue': np.sum})

# make the column names more meaningful
cohorts.rename(columns={'uuid': 'TotalUsers'}, inplace=True)
cohorts.head()

,,TotalUsers,Revenue
CohortGroup,Period,,
00,00,10876,377345.51


In [ ]:
#Label the CohortPeriod for each CohortGroup
def cohort_period(df):
    """
    Creates a `CohortPeriod` column, which is the Nth period based on the user's first gift.
    
    Example
    -------
    Say you want to get the 3rd month for every user:
        df.sort(['UserId', 'OrderTime', inplace=True)
        df = df.groupby('UserId').apply(cohort_period)
        df[df.CohortPeriod == 3]
    """
    df['CohortPeriod'] = np.arange(len(df)) + 1
    return df

cohorts = cohorts.groupby(level=0).apply(cohort_period)
cohorts.head()

In [ ]:
#User Retention by Cohort Group
# reindex the DataFrame
cohorts.reset_index(inplace=True)
cohorts.set_index(['CohortGroup', 'CohortPeriod'], inplace=True)

# create a Series holding the total size of each CohortGroup
cohort_group_size = cohorts['TotalUsers'].groupby(level=0).first()
cohort_group_size.head()

In [ ]:
cohorts['TotalUsers'].unstack(0).head()

In [ ]:
user_retention = cohorts['TotalUsers'].unstack(0).divide(cohort_group_size, axis=1)
user_retention.head(10)

In [ ]:
#output cohort
current_time = datetime.now()
info = current_time.strftime('%m%d%Y')
output_filename = ("cohort_" + str(info[:4]) + str(info[6:]) + ".csv")
cohorts.to_csv(output_filename, sep=',', header=True)

In [ ]:
#output user rentention
current_time = datetime.now()
info = current_time.strftime('%m%d%Y')
output_filename = ("user_retention_" + str(info[:4]) + str(info[6:]) + ".csv")
user_retention.to_csv(output_filename, sep=',', header=True)

In [ ]:
sns.set(style='white')
plt.figure(figsize=(15, 15))
plt.title('Cohorts: User Retention')
sns.heatmap(user_retention.T, mask=user_retention.T.isnull(), annot=True, fmt='.0%');

In [ ]:
user_retention[['2010', '2011', '2012','2013', '2014', '2015','2016', '2017', '2018','2019']].plot(figsize=(10,5))
plt.title('Cohorts: User Retention')
plt.xticks(np.arange(1, 12.1, 1))
plt.xlim(1, 12)
plt.ylabel('% of Retention');